In [ ]:
%load_ext autoreload
%autoreload 2

# 0. Intro: deepglue demo
This demo will highlight some of the features of deepglue using the [fashion mnist](https://github.com/zalandoresearch/fashion-mnist) dataset for classification. It includes 70k pictures of different articles of clothing sorted into ten categories (e.g., t-shirts). 

Let's import libraries and set up a logger. 

In [ ]:
import gdown
from IPython.display import HTML
import logging
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from PIL import Image
from pprint import pprint
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.transforms.v2 as transforms
from torchvision.transforms.v2 import ToImage, ToDtype

import umap
import zipfile

import deepglue as dg

We'll set the logger to just print warnings for now. If you run into problems you can set the logging level to `DEBUG`.

If you don't want your world clutter with logging, you can delete the following or comment it out.  

In [ ]:
log_format = "%(asctime)s - %(filename)s - %(funcName)s - line %(lineno)d - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.WARNING,  # You can set this to the desired logging level INFO DEBUG WARNING etc
                    format=log_format)  

In what follows, you can explore any function's documentation by typing the function name followed by a question mark. E.g.,  just type in `dg.validate_one_epoch?` in a code cell to get a printout of the documentation for `dg.validate_one_epoch()`. To get a deeper dive into a deepglue function, you can go to the source code at [the repo](https://github.com/EricThomson/deepglue) or the [online documentation](https://deepglue.readthedocs.io/en/latest/). For instance, you can explore how `validate_one_epoch()` works at the [online documentation for the function](https://deepglue.readthedocs.io/en/latest/api/#deepglue.training_utils.validate_one_epoch). Also, you can ask questions at the repo's [Discussion forum](https://github.com/EricThomson/deepglue/discussions). If you find a problem, please [raise an issue](https://github.com/EricThomson/deepglue/issues). 

# 1. Set up directory structure 
It's key to be organized for any deep learning project. In what follows we will set up a directory structure for the project. 

## Set up directory for all projects
Since this is the first demo, we'll set it up the default projects directory where all deepglue projects will be stored. 

The following will set up projects in `~/.deepglue/projects/` (Linux/Mac) or `C:/Users/Username/.deepglue/projects/` (Windows). 

In [ ]:
# Feel free to change the following defaults, this is just for convenience to get things started.
deepglue_dir = Path.home() / ".deepglue"
projects_dir = deepglue_dir / "projects"

try:
    projects_dir.mkdir(parents=True, exist_ok=False) # prevent overwriting
except FileExistsError:
    print("'projects/' directory already exists. Skipping creation.\n")

print(f"Your deepglue projects directory is: {projects_dir}")

## Directory for fashion project
Now let's create the fashion project directory and minimal structure for our fashion mnist classification project. Given a new project name (`fashion`), deepglue's `create_project()` function creates the following directory structure within `projects/`:

    fashion/
        data/
        models/

This is a kind of minimal structure for a deep learning project. You can add more as needed.

In [ ]:
project_name = "fashion"
project_dir, project_data_dir, project_models_dir = dg.create_project(projects_dir, project_name)

print(f"Your new project directory: {project_dir}")
print(f"Datasets will go into {project_data_dir}")

# 2. Download data
Our slightly modified fashion-mnist dataset (modified to be rgb just to make some things simpler), is in the cloud in a zip file. Let's download and unzip the data into the `project_data_dir` we already created.

In [ ]:
data_url = r'https://drive.google.com/uc?id=1B15ViE9lKquepM2TCvApe9gGPeQIxiUS'
filename = 'fashion_mnist.zip'
data_zip_path = project_data_dir / filename
data_dir = project_data_dir / 'fashion_mnist' # root data dir for project
print(f"Will attempt download to {data_zip_path}")

In [ ]:
%%time
if not data_zip_path.exists():
    print("Downloading...may take few seconds")
    gdown.download(data_url, str(data_zip_path), quiet=True, fuzzy=True)
else:
    print(f"{filename} already downloaded. Download skipped.")

Let's unzip the compressed data using zipfile's `extractall()`. The following has some extra wrinkles to avoid errors and repeating decompression. 

In [ ]:
%%time
if data_dir.exists():
    print("Already extracted. Skipping.")
else:
    if data_zip_path.suffix == '.zip':
        print(f"Unzipping to {project_data_dir}\nLots of files, so it takes a while...")
        with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
            zip_ref.extractall(project_data_dir)
        print("Done unzipping!\n")
    else:
        print('Not a zip file.')

train_dir = data_dir / 'train'
valid_dir = data_dir / 'valid'
test_dir = data_dir / 'test'

Our final directory structure follows a standard that is used by several functions in `pytorch`:

        projects/
            fashion/
                models/
                data/
                    fashion_mnist/
                        train/
                            0/  [tshirt]
                            1/  [trouser]
                        valid/
                            0/
                            1/   
                        test/
                            0/
                            1/

Each end node in this directory tree (`0/`) contains image data from the relevant category: feel free to navigate to the relevant spots on your machine to check out the data. Below we will show how to use `deepglue` to inspect random samples of the data. 

Our mapping from subdirectory names to actual categories is as follows:

In [ ]:
category_map = {'0': 'tshirt',
                '1': 'trouser',
                '2': 'pullover',
                '3': 'dress',
                '4': 'coat',
                '5': 'sandal',
                '6': 'shirt',
                '7': 'sneaker',
                '8': 'bag',
                '9': 'ankle_boot'}

categories = ['0','1','2','3','4','5','6','7','8','9']
category_names = [category_map[key] for key in categories]

<div class="alert alert-info">
<h3 style="margin: 1px 0 6px 0;">Scalable data structures</h3>
In real-world projects there will often be multiple datasets placed in the project's top-level <code>data/</code> directory. Since we are working with a single dataset, we could in theory just extract the <code>train/</code>, <code>valid/</code> and <code>test/</code> folders directly into <code>data/</code> to keep a more flat structure. But our more nested structure mimics how larger projects handle multiple datasets in a scalable way, so we'll stick with it.
</div>

# 3. Explore data 
An initial look at the dataset.

Deepglue will plot some random images from the dataset:

In [ ]:
dg.plot_random_sample(data_dir, category_map, split_type='train', num_to_plot=20);

Within the project, the data are divided into training, validation, and testing splits. How many are in each split?

In [ ]:
num_per_split = dg.count_by_split(data_dir)

total_samples = num_per_split['train'] + num_per_split['valid'] + num_per_split['test']
print(f"Num samples total: {total_samples}")
pprint(num_per_split)
data_splits = ['train', 'valid', 'test']
proportion_per_split =  [num_per_split['train']/total_samples, 
                         num_per_split['valid']/total_samples, 
                         num_per_split['test']/total_samples]

# Plot number in train/validation/test splits
plt.bar(data_splits, proportion_per_split)
plt.title("Proportion per split")
plt.xlabel('Split Type')
plt.ylabel('Proportion');

We have 70k total samples, with 50k set aside for training, and 10k for validation and testing respectively (70%, 15%, 15%). We can check out how many are in each category, within each split:

In [ ]:
num_category_by_split = dg.count_category_by_split(data_dir)

That creates a dict with the splits as the keys (`train`, `valid`, and `test`), and each contains the number in each category in a dictionary:

In [ ]:
print('Training split counts:')
pprint(num_category_by_split['train'])

Let's plot the proportion represented in the categories in the three data splits:

In [ ]:
# get counts and proportions
train_counts = np.array([num_category_by_split['train'][key] for key in categories])
train_proportions = train_counts/sum(train_counts)

valid_counts = np.array([num_category_by_split['valid'][key] for key in categories])
valid_proportions = valid_counts/sum(valid_counts)

test_counts = np.array([num_category_by_split['test'][key] for key in categories])
test_proportions = test_counts/sum(test_counts)

# plot them
fig, (ax_test, ax_val, ax_train) = plt.subplots(3,1,figsize=(5,5)) # width x height

#train (bottom)
ax_train.bar(category_names, train_proportions)
ax_train.tick_params(axis='x', labelrotation=45)
ax_train.set_title("Training Data")
ax_train.set_xlabel('Category')

# validation (middle)
ax_val.bar(category_names, valid_proportions)
ax_val.set_ylabel('Proportion');
ax_val.set_title("Validation Data")
ax_val.set_xticks([]) 

# test (top)
ax_test.bar(category_names, test_proportions)
ax_test.set_title("Test Data")
ax_test.set_xticks([])

plt.tight_layout()

We can see this is (by design) an extremely balanced data set, in all three splits. 

# 4. Define network model
To learn the fashion classificaiton task, we'll start with a pre-trained [resnet18 model](https://www.run.ai/guides/deep-learning-for-computer-vision/pytorch-resnet) that was trained on the Imagenet 1k dataset (1000 categories with over 1 million total images). It's a relatively simple pre-trained model to use as a base for transfer learning. We are initially freezing all the model parameters, and then will unfreeze the final two convolution layers, and replace the fully connected (fc) layer. So we'll be keeping the early layers which extract basic features, but allowing the later layers to learn the higher-level fashion-specific features in fashion-mnist. 

Feel free to substitute a different model and adapt the code to suit your needs. For instance, `resnet50` will almost certainly perform better, but it will also take up way more memory and take longer to train. So for this little demo, we're sticking with `resnet18` (which also performs very well). We'll discuss the topic of trying out different models at the end in the `Test data` section.

In [ ]:
num_classes = len(category_map)  # Adjust to match the number of classes in Fashion MNIST
num_fc_hidden_units = 256  # Number of hidden units in fully connected layer: feel free to tweak

In [ ]:
# Load pre-trained ResNet50 model
resnet_weights = models.ResNet18_Weights.IMAGENET1K_V1
resnet18 = models.resnet18(weights=resnet_weights)

# Freeze all model parameters initially
for param in resnet18.parameters():
    param.requires_grad = False
    
# unfreeze blocks 3 and 4
for param in resnet18.layer3.parameters():
    param.requires_grad = True
    
for param in resnet18.layer4.parameters():
    param.requires_grad = True

# Get the number of inputs for the original fully connected (fc) layer
num_fc_inputs = resnet18.fc.in_features

# Replace the final fully connected layer (note requires_grad is True by default for a brand-new layer)
resnet18.fc = nn.Sequential(nn.Linear(num_fc_inputs, num_fc_hidden_units),  # Projection from backbone to hidden units
                            nn.ReLU(),                                      # Activation for non-linearity
                            nn.Linear(num_fc_hidden_units, num_classes))    # Final layer for class prediction

We'll also define the loss function and optimizer (which includes the learning rate schedule) for the network.

In [ ]:
loss_func = nn.CrossEntropyLoss() 
optimizer = optim.Adam(resnet18.parameters(), lr=0.0001)  

<div class="alert alert-info">
<h3 style="margin: 1px 0 6px 0;">Expected inputs for resnet</h3>
Resnet models were trained on 224 x 224 images, and this matters. If you try to feed the model different sized images, it will not perform as well, or you may get errors at some point in your project. Fashion mnist images are 28x28, so in what follows we will upsample them to 224x224. We'll also convert the inputs to RGB. We could define a completely new network and train from scratch, but then we'd lose out on the gains from the pretrained network. 
</div>

# 5. Set up data for training
Having a bunch of data in folders is great, but `torchvision` provides lots of utilities to make funneling such data through training pipelines really easy. Also, integrating transformations like random cropping, rotations, and normalization directly into the our pipelines simplifies training, and `torchvision` has a great api that integrates such augmentations directly into the data pipeline. We'll start there.

## Transforms
Torchvision has an amazing set of transforms that apply whether you are doing classification, object detection, or scene segmentation. To learn more about the transforms, see their [documentation](https://pytorch.org/vision/main/transforms.html#v2-api-ref) or [example page](https://pytorch.org/vision/main/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py). 

We are going to set up a relatively simple transform here just to show the logic. First we'll define a couple of transforms that we will randomly apply in our transform function:

In [ ]:
gaussian_noise = transforms.GaussianNoise(mean=0, # mean of sampled noise
                                         sigma=0.1, # std of sampled noise
                                         clip=True)  # clip to [0,1] after adding noise
gaussian_blur = transforms.GaussianBlur(kernel_size=(7,7), # kernel size (width, height)
                                        sigma=(0.8, 0.8)) # sigma: min, max randomly chosen

Then define a function for a data transform that can be applied to different data splits. 

In [ ]:
def data_transform(train=False):
    """
    Define data transformation to be used when data is ingested for use by datasets.
    """
    transform_pipeline = []
    
    transform_pipeline.append(transforms.Resize((224, 224)))     # resnet expect this size
    transform_pipeline.append(transforms.ToImage())  # Convert PIL image to tensor: many transforms only work on torch tensor
    transform_pipeline.append(transforms.ToDtype(torch.float32, scale=True)) # many transforms only work with float
    transform_pipeline.append(transforms.RGB()) # resnet expects RGB (if it is already RGB, nothing changes)

    if train:
        transform_pipeline.append(transforms.RandomApply([gaussian_noise], p=1/2)) # will apply p of the time
        transform_pipeline.append(transforms.RandomApply([gaussian_blur], p=1/2))  
        transform_pipeline.append(transforms.RandomHorizontalFlip(p=1/2))
        transform_pipeline.append(transforms.RandomRotation(20, fill=0.445, expand=False))  # will rotate (-20,20); expand would resize so image fits in image shape
        
    # Normalize to standard values for resnet
    transform_pipeline.append(transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                   std=[0.229, 0.224, 0.225]))  
    return transforms.Compose(transform_pipeline)

In the above, we have defined a general transform function that we can define for diffrent data splits: those that are set for training data will have distortions applied to the data for augmentation purposes (gaussian blur, noise, flips, rotations). Those for validation and test will only have "nondistorting" changes applied: they will be rescaled to the proper size, changed to torch tensors, converted to floats, etc. 

### Transform demo
Let's look at how the transform works in the case of training data just for fun. We'll set train to `True` so we'll see the effects of the augmenting transforms. 

In [ ]:
demo_transform = data_transform(train=True)

Let's get a random image from our dataset:

In [ ]:
category_to_plot = '7'
demo_image_path, demo_image_category = dg.sample_random_images(data_dir, 
                                                               category_map, 
                                                               category=category_to_plot) 
demo_image = np.array(Image.open(demo_image_path[0]))
dg.plot_transformed(demo_image, demo_transform, num_to_plot=9);
print(category_map[category_to_plot])

You can see that we get multiple views of the sneaker, some are pretty distorted. This is good! We want to give the network some tricky instances for training. 

### Define transforms for our datasets
Let's define three different image transforms that will be used by our dataset pipeline when we load data from disk: 

In [ ]:
image_transforms = {'train': data_transform(train=True),
                    'valid': data_transform(train=False),
                    'test': data_transform(train=False)}

## Datasets
Let's create our training, validation, and test datasets using torchvision's `datasets.ImageFolder` class. This is a convenience class that hooks into image folders that are structured the way we have set them up. In general, pytoch datasets provide a flexible way to define how your data is loaded, transformed, and accessed, making it highly useful for deep learning workflows. The primary use of datasets is get wrapped into a `DataLoader`, which will handle batching, shuffling, and parallel data loading using multiple workers. Datasets are designed to load data lazily, meaning they load individual items on-the-fly when accessed: this minimizes memory usage, which is especially helpful fo large datasets. 

We'll create a dictionary of datasets, one dataset for each data split. 

In [ ]:
data = {'train': datasets.ImageFolder(root=train_dir, transform=image_transforms['train']),
        'valid': datasets.ImageFolder(root=valid_dir, transform=image_transforms['valid']),
        'test': datasets.ImageFolder(root=test_dir, transform=image_transforms['test'])}

## Dataloaders
The point of the above is to create data loaders, which is what we actually use directly during training. They shuffle the data, create batches, and parallelize things with multiprocessing to make everything go faster. 

Dataloaders require a couple of parameters and you might have to adjust these depending on your system. Just to review some ML terminology: one *epoch* is a single run through the entire data set. Our training data has 50k images, so one epoch of the training data will be a run through 50k images. This is way too much to run through all at once. A *batch* is the number of images in a subset that is run for processing in each loop during training/validation. The error is calculated for this subset and the network is updated during training for each batch. `num_workers` is the number of (CPU) processes that will work to generate batches in parallel: it can significantly speed up runtime. 

In [ ]:
batch_size = 256   # 128 on laptop, 256 on workstation
num_workers = 6   # 4 on laptop, 6 on workstation

# persistent workers makes things startup faster, but only if you have multiple workers
if num_workers > 0:
    persist_workers = True
else: 
    persist_workers = False

In [ ]:
train_data_loader = DataLoader(data['train'], 
                               batch_size=batch_size, 
                               shuffle=True,
                               num_workers=num_workers,
                               drop_last=True,  # drop dangling batch at end
                               persistent_workers=persist_workers)

valid_data_loader = DataLoader(data['valid'], 
                               batch_size=batch_size, 
                               shuffle=True,
                               num_workers=num_workers,
                               persistent_workers=persist_workers)

test_data_loader = DataLoader(data['test'], 
                              batch_size=batch_size, 
                              shuffle=True,
                              num_workers=num_workers,
                              persistent_workers=persist_workers)

<div class="alert alert-info">
<h3 style="margin: 1px 0 6px 0;">Multiprocessing on different operating systems</h3>
Different systems can take extra time to create multiple workers. Windows and (newer) Macs uses a different method than Linux to create new workers. On some older Windows systems, setting <code>num_workers</code> to any number greater than <code>0</code> will simply cause your system to hang. If this happens, you unfortunately have to set <code>num_workers = 0</code>. However, do give it a minute to start, because once the workers are set up initially, the speed payoff is <em>significant</em>.
</div>

If you have already trained a model, you can jump to step 7 -- load the network.

# 6. Train the network
We'll use deepglue's `train_and_validate()` to run through the training data (50k images) and validation data (10k images). 

There are a couple of parameters here to set. How many times will we cycle throught the datasets during training (`num_epochs`), and the `topk` accuracy values to return. Top-k prediction counts as correct if the prediction was in the top k highest probabilities from the network (e.g., if the network's highest estimates for an image were `[sneaker, sandal, ankle_boot]`, for a `sandal` then it was top-3 accurate). This is useful for datasets with lots of similar categories. 

We are training on four epochs because initial exploration showed validation accuracy started to dip around this point (overfitting), and we want this demo to not take too long to run. 

In [ ]:
topk = (1,3)
num_epochs = 3

Note the following can take a long time (many minutes), especially on a laptop. As mentioned above, it can take a while to get started as it sets up the workers. 

In [ ]:
#  reset our logging level so we can get some feedback during training
logging.getLogger().setLevel(logging.INFO)

trained_model, train_history = dg.train_and_validate(resnet18,
                                                     train_data_loader,
                                                     valid_data_loader,
                                                     loss_func,
                                                     optimizer,
                                                     device='cuda',
                                                     topk=topk,
                                                     epochs=num_epochs);

Under the hood, `dg.train_and_validate()` works by calling `dg.train_one_epoch()` and `dg.validate_one_epoch()`, each `num_epochs` times.  `dg.train_one_epoch()` is the main workhorse for training models. It takes in the model, data loader (and a few other things), and steps through training for the required number of batches to train through all the data for an entire epoch. If you want to understand how that works, it can be instructive to step through the code. 

If you run out of GPU memory, try decreasing batch size and number of workers. On a laptop with 4GB of GPU RAM, 8 CPUs, the above cell took about 15 minutes.

## View loss/accuracy
Let's see how it did during training. Note we will explore other metrics below this is just to get a quick sense for how things went. 

In [ ]:
epoch_array = np.arange(num_epochs)

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(10,4))

# Plot loss 
ax_loss.plot(epoch_array, train_history['train_loss'], color='blue',  label="Training Data", marker='.')
ax_loss.plot(epoch_array, train_history['val_loss'], color='firebrick', label="Validation Data", marker='.');
ax_loss.set_title('Training Loss')
ax_loss.set_ylabel('Loss')
ax_loss.set_xlabel('Epoch')
ax_loss.legend();

# Plot topk accuracies
ax_acc.plot(epoch_array, train_history['val_topk_accuracy'][:,1], color='firebrick',  label="Validation top 3", marker='.');
ax_acc.plot(epoch_array, train_history['train_topk_accuracy'][:,1], color='blue', label="Training top 3", marker='.')
ax_acc.plot(epoch_array, train_history['val_topk_accuracy'][:,0],  color='darksalmon', label="Validation top 1", marker='.');
ax_acc.plot(epoch_array, train_history['train_topk_accuracy'][:,0],  color='lightsteelblue', label="Training top 1", marker='.')

ax_acc.axhline(y=100, color='k', linestyle='--', linewidth=0.5)
ax_acc.set_title('Training Accuracy')
ax_acc.set_ylabel('Accuracy')
ax_acc.set_xlabel('Epoch')
ax_acc.legend();

plt.tight_layout()

Things look reasonable on a first pass. We are inspecting to see if validation loss/accuracy starts to increase/decrease while training loss continues to decrease/increase. This would be a classic sign of overfitting. 

# 7. Save the network
Docs on this: https://pytorch.org/tutorials/beginner/saving_loading_models.html 

Temporary until we have proper checkpoint save/load built into training function.

In [ ]:
checkpoint_save_name = r"resnet18_final.pth" #
checkpoint_save_path = project_models_dir / checkpoint_save_name
checkpoint_save_path

In [ ]:
torch.save({'model': trained_model,   # Model parameters
            'optimizer': optimizer,  # Optimizer parameters
            'epochs': num_epochs,  
            'train_loss': train_history['train_loss'],
            'val_loss': train_history['val_loss'],  
            'train_accuracy': train_history['train_topk_accuracy'],
            'val_accuracy': train_history['val_topk_accuracy'],
            'topk': topk,}, checkpoint_save_path) 

## Load network (optional)
To save time, you can skip from Step 1 to this step once you have trained the network once. 

In [ ]:
load_model = False
if load_model:
    # initialize things
    print("Loading final model checkpoint")
    checkpoint_load_name = r"resnet18_final.pth" #
    checkpoint_load_path = project_models_dir / checkpoint_load_name
    device = 'cuda'
    if checkpoint_load_path.exists() and torch.cuda.is_available():
        print("model exists, cuda available.")

    # load the data
    final_checkpoint = torch.load(checkpoint_load_path, weights_only=False)

    # Unpack values you want (TODO: cut some of these you don't use)
    trained_model = final_checkpoint['model']
    optimizer = final_checkpoint['optimizer']
    num_epochs = final_checkpoint['epochs']
    train_loss = final_checkpoint['train_loss']
    val_loss = final_checkpoint['val_loss']
    train_topk_accuracy = final_checkpoint['train_accuracy']
    val_topk_accuracy = final_checkpoint['val_accuracy']
    topk = final_checkpoint['topk']
else:
    print("Not loading model -- likely a training run")

# 8. Check model performance
First let's visually inspect model performance over some random images from the validation data set. Then we'll look at some metrics over the entire validation dataset.

## Inspect some predictions
We'll use a few deepglue convenience functions to get a few random images and predict their identity using the trained model:

In [ ]:
rand_paths, rand_categories = dg.sample_random_images(data_dir, 
                                                      category_map, 
                                                      split_type='valid', 
                                                      num_images=10)
random_stack = dg.load_images_for_model(rand_paths, 
                                        data_transform(train=False)); # nondistorting transform
predicted_probs = dg.predict_batch(trained_model, random_stack); # defaults to 'cuda' device

Using `dg.plot_prediction_grid()`, we'll plot the actual image and top prediction on the left, and the `top_n` predictions with their probabilities on the right: let's check out the top five predictions of the model. 

In [ ]:
dg.plot_prediction_grid(random_stack, 
                        predicted_probs, 
                        rand_categories, 
                        category_map, 
                        top_n=5, 
                        figsize_per_plot=(2, 2), 
                        logscale=True);

## Metrics
While images are good (and we get good performance on individuals), we should examine performance over the entire validation data set (10k images). 

Scikit learn has many metrics we can use. We have already built a dataloader for validation data above, and deepglue has a function for predicting all the data given a dataloader and the trained model.

In [ ]:
val_preds, val_labels, val_probs = dg.predict_all(trained_model, valid_data_loader);

Now, with the predictions, correct labels, we can get lots of metrics. We'll focus on some of the basic metrics for now. 

In [ ]:
val_preds.shape

### Confusion matrix
The confusion matrix is very helpful as a visual display: it tells you the count of the actual category and predicted category for all ten categories. 

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cm_disp = ConfusionMatrixDisplay.from_predictions(val_labels,
                                                  val_preds,
                                                  display_labels=category_names,
                                                  xticks_rotation=45.,
                                                  cmap='magma');

### Classification report
Scikit learn generates a useful classification report that tells you the precision (indicates levels of false positives along the column of the confusion matrix) and recall (indicates levels of false negatives along the row of the confusion matrix) for each category. F1 is a combination of both precision and recall.  

The classification report also aggregates these measures into overall accuracy (overall proportion correct), average precision and recall (called 'macro average'), and weighted average (weighted by class size, which would be useful for imbalanced data). 

For more on the classification report, there is a useful discussion here: https://www.nb-data.com/p/breaking-down-the-classification. 

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(val_labels, val_preds, target_names=category_names);
print(report)

You can see that some of the categories are classified nearly perfectly (trousers, bags). While others are really tough, in particular shirts. Is this because our network needs more training, resnet18 is not up to the task, or maybe something intrinsic to our dataset? 

Before investing a ton of time tweaking parameters, let's do some visualization of our data and feature space to find the error patterns. 

# 9. Visualize clustering in feature space
As just mentioned, while our network is performing very well, we have a decision to make. Should we tweak some parameters? Should we bring in some more fancy learning rate scheduler? Maybe train for more epochs? Some more heavy augmentation in our transformer might be helpful. 

These are all reasonable options, but this notebook is meant to demo deepglue basics, not deep learning weeds. Also, sometimes it is helpful to visualize what's happening inside of a network before spending that 80% of your time eking out that 2% improvement in the model performance. Pytorch provides some useful tools to extract the activity (features) inside networks, and deepglue has utilities for visualizing the features embedded there. 

## Feature extraction
The way feature extraction works is run the image(s) through the network, and extract the activity patterns in the desired layer(s) for each image. 

During normal network operation, we only care about the final prediction of the network, that's what is returned as output from the network. But for analysis of the internals of the network, `pytorch` provides internal hooks to the nodes as a convenience function using `create_feature_extractor()`. We can feed this function images and the network will return the activation values from *any node* of the network when given those images. You feed it batches through a data loader, give it the full data set, but then you've cracked open the black box and can visualize the internal operations of the network.

We first need to create a data loader for the feature extractor: deepglue's `prepare_ordered_data()` creates a dataloader for feature extraction that will go in order through all the data in a data split, in order, and return the data loader and the image paths: 

In [ ]:
image_paths, ordered_loader = dg.prepare_ordered_data(data_dir, 
                                                      image_transforms['valid'], 
                                                      num_workers=4, 
                                                      batch_size=56, 
                                                      split_type='valid')

We have to decide which layer(s) you want to extract features from. 

In torchvision, the convention is that the variable `return_nodes` defines the layers from which features can subsequently be extracted (keys are the actual layer names, and values are the names we give them for access later). As discussed above, we typically just want outputs returned from the final output nodes, and this lets us get data from anywhere in the network.

In [ ]:
return_nodes = {'relu': 'cnn_features',                  # Early traditional CNN features
                'layer1.1.relu': 'resnetL1_features',
                'layer2.1.relu': 'resnetL2_features',
                'layer3.1.relu': 'resnetL3_features',
                'layer4.1.relu': 'resnetL4_features'}
pprint(return_nodes)

To list all of the options for extracting features you could run the following code (expect a long printout):

    for name, module in trained_model.named_modules():
        print(name)

We are going to keep it simple and just examine the activity very late to visualize clustering of higher level abstract features. Note if you were using `resnet50`, there would be *many* more layers to choose from.

Note the earliest one (relu) is from the first convolutional neural network that don't include any residual blocks, so it's considered a traditional feature extraction layer that would provide access to very low-level features (beware there would be *lots* of dimensions so you would needs lots of RAM). The other four are the outputs of the four residual blocks (we expect higher-level fashion features in layer 4 so will extract those in what follows).

We use the built-in `torchvision` function `create_feature_extractor()` to create the feature extractor: it just needs our trained model and dictionary of return nodes.

In [ ]:
trained_model.eval();
feature_extractor = create_feature_extractor(trained_model.to('cuda'), 
                                             return_nodes=return_nodes)

Run feature extraction on chosen layer. Note this can take a while because it is running inference on the entire validation dataset so it can extract the activation patterns from the selected layer. 

In [ ]:
layer = 'resnetL4_features'

In [ ]:
features, labels = dg.extract_features(ordered_loader, 
                                       feature_extractor, 
                                       layer=layer,
                                       device='cuda')

The features extracted are fairly high dimension (over 25k dimensions). This is actually much lower than if we had extracted features from earlier layers in the network. 

In [ ]:
features.shape, labels.shape

## Dimensionality reduction and visualization
It is hard to visualize tens of thousands of features. We will now use umap to project these high-dimensional activity patterns to a 2d space to visualize how much they are clustering according to category. We will render this using static plot (Matplotlib) and dynamically (using Bokeh), so when you hover over the scatter points it shows the image, so you can see how the points between clusters look. 

The following visualizations are adapted from a umap demo: https://umap-learn.readthedocs.io/en/latest/basic_usage.html 

In [ ]:
umap_model = umap.UMAP(n_components=2, 
                       n_epochs=200, 
                       low_memory=True,
                       verbose=True)

UMAP is itself an iterative ML algorithm, so the following will take a while:

In [ ]:
%%time
layer_features_umap = umap_model.fit_transform(features)

<div class="alert alert-info">
<h3 style="margin: 1px 0 6px 0;">Other dimensionality reduction techniques</h3>

If you want to compare UMAP to PCA or other dimensionality reduction techniques, it should flow nicely through in the code. For instance:    
<code>
pca = PCA(n_components=2)
layer_features_pca = pca.fit_transform(features)
</code>
    
You can then just replace `layer_features_umap` with `layer_features_pca` in the code. PCA is much faster, and will project based just on variance which is easier to interpret (the clusters won't look as nice though). 
</div>

### Static visualization
Let's first do a static visualization of the feature clusters using matplotlib

In [ ]:
plt.figure(figsize=(5, 5))

# Scatter plot of the subsampled features
scatter = plt.scatter(
    layer_features_umap[:, 0],  # x-coordinates from UMAP
    layer_features_umap[:, 1],  # y-coordinates from UMAP
    c=labels,           # Color code by labels
    cmap=plt.cm.tab10,              
    s=10,                          
    alpha=0.7                      
)

# Adding a colorbar for the labels
colorbar = plt.colorbar(scatter, boundaries=np.arange(num_classes + 1) - 0.5)
colorbar.set_ticks(np.arange(num_classes))
colorbar.set_ticklabels(category_names)
plt.xticks([])
plt.yticks([])

# Title and labels
plt.title('UMAP Projection of Fashion')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')

# Show plot
plt.axis('equal')
plt.grid(True)
plt.show()

We can see patterns that match what we saw with our classification report: trousers and bags clearly distinct, shirts overlapping with other categories. However, it would be nice to have more details. For instance, which of those spots were correct/incorrect? What do the shirts actually look like that are close to coats? Let's build an interactive visualization tool that will give us such details.

### Interactive feature visualization
We will build an interactive scatter plot that plots a little embeddable image sprite when we hover over a point in the plot:

In [ ]:
ind_to_embed = 333 
encoded_image_str = dg.create_embeddable_image(image_paths[ind_to_embed], size=(60,60))
HTML(f'<img src="{encoded_image_str}" />')

Is the above a shirt or a tshirt? What criteria were used for these categories anyway? These are things we can dig into a bit in what follows. To get predictions and error patterns that map onto the ordered data loader used for feature extraction, we need to get predictions from the ordered data loader:

In [ ]:
ordered_predictions, ordered_labels, _ = dg.predict_all(trained_model, ordered_loader);

In [ ]:
print(f"BTW, the above was a {category_names[ordered_labels[ind_to_embed]]}")
print("How did the network do?")
print(f"Actual category: {ordered_labels[ind_to_embed]}: Predicted category {ordered_predictions[ind_to_embed]}")

We can use deepglue's `plot_interactive_projection()` to plot everything we wanted in one plot:
- Scatter plot of the features projected in umap space, where hovering shows you see the sprite of the corresponding image.
- o's show correct predictions, x's show incorrect predictions so you can see what features confused our network.
- You can interact with the plot by zooming, selecting regions that you are interested in (see selection tools on right to determine whether you wheel zoom or box select).

In [ ]:
dg.plot_interactive_projection(layer_features_umap, 
                               ordered_labels.numpy(), 
                               image_paths, # to show sprites
                               category_map,
                               predictions=ordered_predictions.numpy(), # to differentiate correct/incorrect
                               title='UMAP Projection', 
                               image_size=(75, 75), # size of sprite popups
                               plot_size=500, # x/y dims of plot
                               show_in_notebook=True) # if not True, will pop up plot in new tab

With this visualization we can start to see the range of visual features that are important for each category, and how they start to blend into each other and make category membership determination very hard (even for us). 

# 10. Test data

So far, we have purposely held off analyzing the test data. We really want to avoid data leakage, so keep the test data sealed off until we are sure we have the final comparisons we want to make. 

Once you have put your final candidate model (or models) through the ringer, and narrowed things down to a small set of models (e.g., your best resnet, vgg, whatever, with the best augmentation and learning rate schedulers), we put them to the final test with your test data to see which generalizes best outside of all of our training regime (which includes training and validation splits). At that point it's time to get a fair estimate of how well the model generalizes outside all of your training data. It probably won't be as good as for the validation data, which we used to tweak the model. That's why we set aside the test data and leave it in a lock box until the very end. 

I'm purposely leaving it out of this notebook to give people a chance to try different models (e.g., `resnet50`), add different augmentations, tweak things, improve the model, before doing this. You can then apply the same metrics we went over above with validation data (accuracy, recall, etc) to the test data and see how well things generalized. 

This is left as an exercise for the reader.